<a href="https://colab.research.google.com/github/danimelsz/phrynomedusa/blob/main/fastqscreen_hDNA_pvanzolini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# hDNA de Phrynomedusa vanzolini

De acordo com a análise de M. Lyra:
- Total number of reads: 63,829,658	
- Number of reads after trimming > 21 bp: 60,786,476

De acordo com a minha análise
- Total number of reads: 63,829,658	
- Number of reads after trimming Illumina adapters and keeping >21 bp: 60,735,261
- Number of reads after trimming PCR duplicates: 57,576,851
- Number of reads after removing human contaminants: 56,733,038 
- Number of reads after removing E. coli contaminants:
- Number of reads after removing mouse contaminants:

Passo-a-passo
1. **FASTQ** explora dados brutos
2. **CUTADAPT** trima reads menores que 21bp e sem adaptadores Illumina
3. **TALLY** trima duplicatas de PCR
4. **FASTQ-SCREEN** trima reads com DNA contaminante de humano, rato, bactéria, etc.
5. **MITObim** realiza mapeamento iterativo contra o mitogenoma completo de *Pithecopus megacephalus*, podendo ser necessário sub-amostrar o pool de reads usando **downsamplig.py** quando há >10 milhões de reads por limitações computacionais
6. **TABLET** permite visualizar a cobertura

Atenção: esse tutorial deve ser readaptado se for rodado em outra conta do Drive que não dani_ymn@alumni.usp.br, pois os diretórios vão mudar.

Atenção 2: Não rode todas as células de uma vez. Alguns passos de edição de arquivo são executados localmente.


# 1 Conda

In [1]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2022-05-23 02:48:59--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2022-05-23 02:48:59--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 37.7M 1s
    50K .......... .......... .......... .......... ..........  0%

In [2]:
!which conda # should return /usr/local/bin/conda


/usr/local/bin/conda


In [3]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cffi-1.14.6                |   py36h400218f_0         224 KB
    ruamel_yaml-0.15.100       |   py36h27cfd23_0         268 KB
    brotlipy-0.7.0             |py36h27cfd23_1003         349 KB
    pyopenssl-22.0.0           |     pyhd3eb1b0_0          49 KB
    urllib3-1.26.8             |     pyhd3eb1b0_0         100 KB
    conda-package-handling-1.7.3|   py36h27cfd23_1         946 KB
    zlib-1.2.12                |       h7f8727e_2         130 KB
    sqlite-3.38.3              |       hc218d9a_0         1.5 MB
    libstdcxx-ng-11.2.0        |       h1234567_0         6.1 MB
    pycosat-0.6.3              |   py36h27cfd23_0         107 KB
    ca-certificates-2022.4.26  |       h06a4308_0  

cffi-1.14.6          |  224 KB | ########## | 100% 
ruamel_yaml-0.15.100 |  268 KB | ########## | 100% 
brotlipy-0.7.0       |  349 KB | ########## | 100% 
pyopenssl-22.0.0     |   49 KB | ########## | 100% 
urllib3-1.26.8       |  100 KB | ########## | 100% 
conda-package-handli |  946 KB | ########## | 100% 
zlib-1.2.12          |  130 KB | ########## | 100% 
sqlite-3.38.3        |  1.5 MB | ########## | 100% 
libstdcxx-ng-11.2.0  |  6.1 MB | ########## | 100% 
pycosat-0.6.3        |  107 KB | ########## | 100% 
ca-certificates-2022 |  132 KB | ########## | 100% 
charset-normalizer-2 |   33 KB | ########## | 100% 
pycparser-2.21       |   94 KB | ########## | 100% 
setuptools-58.0.4    |  979 KB | ########## | 100% 
tk-8.6.11            |  3.3 MB | ########## | 100% 
tqdm-4.63.0          |   80 KB | ########## | 100% 
cryptography-35.0.0  |  1.5 MB | ########## | 100% 
requests-2.27.1      |   52 KB | ########## | 100% 
ld_impl_linux-64-2.3 |  732 KB | ########## | 100% 
libffi-3.3  

In [4]:
import sys
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [5]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.7/site-packages"))


In [6]:
!conda install --channel conda-forge featuretools --yes


Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [7]:
# Montar o Drive para ter acesso aos arquivos na nuvem
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2 Preprocessing


## 2.1 FASTQC: Dados brutos

Vamos começar fazendo uma análise exploratória dos dados brutos. A contagem de reads resultou em 63829658 (exatamente igual à análise de M. Lyra). O output está localizado em /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/Pvanzo_MZUSP3998_Museum2022_S10_R1_001_fastqc.html

In [ ]:
# Instalar FASTQC 
!conda install -c bioconda fastqc

In [ ]:
# FASTQC dos dados brutos
!fastqc /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz

Started analysis of Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 5% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 10% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 15% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 20% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 25% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 30% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 35% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 40% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 45% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 50% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 55% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 60% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.gz
Approx 65% complete for Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq.

## 2.2 CUTADAPT: trim >21 bp


In [ ]:
# Instalar CUTADAPT
!conda install -c bioconda cutadapt

Vamos agora remover reads menores do que 21 bp. Vamos usar CUTADAPT com os seguintes parâmetros:

- -a CCGGTAC: utiliza o adaptador p5/p7 CCGGTAC
- -m 21: remove reads menores do que 21bp
- -o: indica qual o diretório e o nome do output

Obs. Se usar TROMMOMATIC, note que o comando "trimmomatic SE" é usado no Google Colab, ao passo que a versão local do Linux 18.04 utiliza o comando "TrimmomaticSE". Além disso, veja que a opção ILLUMINACLIP requer que coloquemos um arquivo .FASTA contendo o adaptador (nesse caso, pus p5 e p7 da tabela de M. Lyra).

In [ ]:
# Remove sequências com menos de 21bp e com adaptadores p5/p7 (CCGGTAC)
!cutadapt -a CCGGTAC -m 21 -o /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing1_trim21bp_adapterCCGGTAC/output_pvanzo_adapterCCGGTAC.fastq.gz /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq


This is cutadapt 1.18 with Python 3.6.13
Command line parameters: -a CCGGTAC -m 21 -o /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/output_pvanzo_adapterCCGGTAC.fastq.gz /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq
Processing reads on 1 core in single-end mode ...
Finished in 1561.83 s (24 us/read; 2.45 M reads/minute).

=== Summary ===

Total reads processed:              63,829,658
Reads with adapters:                22,973,264 (36.0%)
Reads that were too short:              32,884 (0.1%)
Reads written (passing filters):    63,796,774 (99.9%)

Total basepairs processed: 4,851,054,008 bp
Total written (filtered):  4,574,600,609 bp (94.3%)

=== Adapter 1 ===

Sequence: CCGGTAC; Type: regular 3'; Length: 7; Trimmed: 22973264 times.

No. of allowed errors:
0-7 bp: 0

Bases preceding removed adapters:
  A: 1.4%
  C: 96.9%
  G: 1.5%
  T: 0.2%
  none/other: 0.0%
    The adapter is preceded by "C" e

Vamos fazer o mesmo procedimento, mas dessa vez removendo o adaptador universal ILLUMINA.

In [ ]:
# Remove reads <21bp e com adaptador universal da Illumina AGATCGGAAGAG
!cutadapt -a AGATCGGAAGAG -m 21 -o /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing2_trim21bp_adapterAGATCGGAAGAG/output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq


This is cutadapt 1.18 with Python 3.6.13
Command line parameters: -a AGATCGGAAGAG -m 21 -o /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/Pvanzo_MZUSP3998_Museum2022_S10_R1_001.fastq
Processing reads on 1 core in single-end mode ...
Finished in 1539.34 s (24 us/read; 2.49 M reads/minute).

=== Summary ===

Total reads processed:              63,829,658
Reads with adapters:                54,318,177 (85.1%)
Reads that were too short:           3,038,368 (4.8%)
Reads written (passing filters):    60,791,290 (95.2%)

Total basepairs processed: 4,851,054,008 bp
Total written (filtered):  2,854,072,670 bp (58.8%)

=== Adapter 1 ===

Sequence: AGATCGGAAGAG; Type: regular 3'; Length: 12; Trimmed: 54318177 times.

No. of allowed errors:
0-9 bp: 0; 10-12 bp: 1

Bases preceding removed adapters:
  A: 33.0%
  C: 16.1%
  G: 17.9%
  T: 33.0%
  none/other: 0.1%

Ov

In [ ]:
# FASTQC do arquivo trimado com reads >21bp e sem adaptadores
!fastqc -o /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing2_trim21bp_adapterAGATCGGAAGAG /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing2_trim21bp_adapterAGATCGGAAGAG/output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz

Started analysis of output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 5% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 10% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 15% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 20% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 25% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 30% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 35% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 40% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 45% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 50% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 55% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 60% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 65% complete for output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz
Approx 70% complete for output2_pvanzo_adapterAGATCGG

## 2.3 TALLY: trim PCR duplicates

In [ ]:
# Instalar TALLY a partir do reaper
!conda install -c bioconda reaper

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - reaper


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    reaper-16.098              |       hed695b0_3         107 KB  bioconda
    ------------------------------------------------------------
                                           Total:         107 KB

The following NEW packages will be INSTALLED:

  reaper             bioconda/linux-64::reaper-16.098-hed695b0_3


Proceed ([y]/n)? y


reaper-16.098        | 107 KB    | : 100% 1.0/1 [00:00<00:00,  3.51it/s]                
Preparing transaction: | done
Verifying transaction: - done
Executing transaction: | done


In [ ]:
# Remove duplicatas mantendo a qualidade PHRED
!tally -i /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing2_trim21bp_adapterAGATCGGAAGAG/output2_pvanzo_adapterAGATCGGAAGAG.fastq.gz -o /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing3_tally/output3_pvanzo_tally.fastq --nozip --with-quality

[tally] data log size 28 (0.25G) hash log size 24 (0.50G) unit size 32
[tally] new chunk (size 268435456, time 9.5, hash fill 0.267, 4484804 sequences)
[tally] new chunk (size 268435456, time 9.0, hash fill 0.534, 8957562 sequences)
[tally] new chunk (size 268435456, time 13.1, hash fill 0.135, 13457184 sequences)
[tally] new chunk (size 268435456, time 13.9, hash fill 0.403, 17940779 sequences)
[tally] new chunk (size 268435456, time 13.6, hash fill 0.663, 22302945 sequences)
[tally] new chunk (size 268435456, time 19.4, hash fill 0.256, 26664941 sequences)
[tally] new chunk (size 268435456, time 18.8, hash fill 0.518, 31052207 sequences)
[tally] new chunk (size 268435456, time 22.2, hash fill 0.108, 35366742 sequences)
[tally] new chunk (size 268435456, time 23.0, hash fill 0.364, 39666143 sequences)
[tally] new chunk (size 268435456, time 23.6, hash fill 0.624, 44018110 sequences)
[tally] new chunk (size 268435456, time 29.6, hash fill 0.212, 48292602 sequences)
[tally] new chunk (s

In [ ]:
!fastqc /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing3_tally/output3_pvanzo_tally.fastq

Started analysis of output3_pvanzo_tally.fastq
Approx 5% complete for output3_pvanzo_tally.fastq
Approx 10% complete for output3_pvanzo_tally.fastq
Approx 15% complete for output3_pvanzo_tally.fastq
Approx 20% complete for output3_pvanzo_tally.fastq
Approx 25% complete for output3_pvanzo_tally.fastq
Approx 30% complete for output3_pvanzo_tally.fastq
Approx 35% complete for output3_pvanzo_tally.fastq
Approx 40% complete for output3_pvanzo_tally.fastq
Approx 45% complete for output3_pvanzo_tally.fastq
Approx 50% complete for output3_pvanzo_tally.fastq
Approx 55% complete for output3_pvanzo_tally.fastq
Approx 60% complete for output3_pvanzo_tally.fastq
Approx 65% complete for output3_pvanzo_tally.fastq
Approx 70% complete for output3_pvanzo_tally.fastq
Approx 75% complete for output3_pvanzo_tally.fastq
Approx 80% complete for output3_pvanzo_tally.fastq
Approx 85% complete for output3_pvanzo_tally.fastq
Approx 90% complete for output3_pvanzo_tally.fastq
Approx 95% complete for output3_pvan

# 3 Contamination

## 3.1 FastQScreen

### 3.1.1 Instalação

In [ ]:
# Instalar FastQScreen
!conda install -c bioconda fastq-screen

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - fastq-screen


The following 

### 3.1.2 Bowtie index de genoma de referência

Por padrão, o FastQScreen realiaz o download de genomas contaminantes mais populares (e.g., humano, rato, E. coli, vetores, etc.), mas não inclui nenhum genoma de anfíbio como referência. Para verificar as reads que são mapeadas contra um genoma de sapo, baixei o arquivo .FASTA da espécie mais aparentada de Phrynomedusa cujo genoma mitocondrial está completo (Pithecopus megacephalus, MG772558). Para usar este genoma do programa FastQScreen, devemos indexá-lo usando bowtie2.

Para gerar os indexados de bowtie2, o arquivo FASTA do mitogenoma de Pithecopus deve estar em /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/fasta/pithecopus_megacephalus_whole_mitogenome_MG772558.fasta. Depois, basta digitar:

In [ ]:
# Criar indexamento de bowtie2 a partir do arquivo FASTA de Pithecopus
!bowtie2-build /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/fasta/pithecopus_megacephalus_whole_mitogenome_MG772558.fasta /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/pithecopus_index
!bowtie2-inspect pithecopus_index

Settings:
  Output files: "pithecopus_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/pithecopus_megacephalus_whole_mitogenome_MG772558.fasta
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 4512
Using parameters --bmax 3384 --dcv 1024
  Doing ahead-of-time memory

Note que os seguintes arquivos de Pithecopus foram criados no diretório /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome.

1.   /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/pithecopus_index.1.bt2
2.   /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/pithecopus_index.2.bt2
3.   /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/pithecopus_index.3.bt2
4.   /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/pithecopus_index.4.bt2
5.   /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/pithecopus_index.rev.1.bt2
6.   /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/pithecopus_index.rev.2.bt2

Agora vamos fazer o mesmo com humanos.

In [ ]:
!bowtie2-build /content/drive/MyDrive/Doutorado/Project/Tutoriais/fastqscreen_reference_genomes/Human/human.fas /content/drive/MyDrive/Doutorado/Project/Tutoriais/fastqscreen_reference_genomes/Human/Homo_sapiens.GRCh38

/bin/bash: bowtie2-build: command not found


In [ ]:
!bowtie2-inspect Homo_sapiens.GRCh38

### 3.1.3 Arquivo de configuração

Existem 3 informações que você precisa descobrir para editar o arquivo fastq_screen.conf:

1.   O caminho do bowtie2
2.   O número de núcleos do processador do Google Colab
3.   O caminho dos genomas contaminantes


In [ ]:
# Verify the path of bowtie2
!which bowtie2

/usr/local/bin/bowtie2


In [ ]:
# Verify the number of threads in Google Colab's CPU
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

2

Os genomas contaminantes foram baixados localmente em http://ftp1.babraham.ac.uk/ftpusr46/FastQ_Screen_Genomes e upados no seguinte diretório: /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/fastq_screen_pvanzo.conf. Para gerar esse arquivo, ele foi editado localmente no nano. Algumas dicas:

- Se a pasta Human estiver em /content/drive/MyDrive/Doutorado/Project/Tutoriais/fastqscreen_reference_genomes

- E se dentro da pasta há os arquivos Homo_sapiens.GRCh38.1.bt2 e Homo_sapiens.GRCh38.2.bt2

- Logo, deve-se digitar 'DATASET Human /content/drive/MyDrive/Doutorado/Project/Tutoriais/fastqscreen_reference_genomes/**FastQ_Screen_Genomes/Human/Homo_sapiens.GRCh38**' dentro do arquivo fastq_screen.conf

```
# Edições necessárias no fastq_screen.conf

...
BOWTIE2 /usr/local/bin/bowtie2
...
THREADS        2
...
DATABASE	Ecoli	/content/drive/MyDrive/Doutorado/Project/Tutoriais/fastqscreen_reference_genomes/E_coli/Ecoli
...
DATABASE    Human    /content/drive/MyDrive/Doutorado/Project/Tutoriais/fastqscreen_reference_genomes/FastQ_Screen_Genomes/Human/Homo_sapiens.GRCh38
...
DATABASE	Adapters	/content/drive/MyDrive/Doutorado/Project/Tutoriais/fastqscreen_reference_genomes/Adapters/Contaminants
...
DATABASE	Pithecopus	/content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/pithecopus_index
```



### 3.1.4 Filtragem

Falta baixar de humano

In [ ]:
# Filtrar com 001 (Ecoli, Adapters, Pithecopus), de modo que apenas reads alinhadas unicamente a Pithecopus permaneçam no outputd
!fastq_screen --tag --filter 003 --aligner bowtie2 --conf /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing4_fastqscreen/fastq_screen_pvanzo.conf /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing3_tally/output3_pvanzo_tally.fastq > /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing4_fastqscreen/output4_pvanzo_fastqscreen

Using fastq_screen v0.14.0
Reading configuration from '/content/output3_pvanzo_tally.tagged_filter.fastq'
User specified filter string of length 3, but 0 genomes specified.
Please adjust parameters.


In [ ]:
!fastqc /content/output3_pvanzo_tally.tagged_filter.fastq

Started analysis of output3_pvanzo_tally.tagged_filter.fastq
Analysis complete for output3_pvanzo_tally.tagged_filter.fastq


## 3.2 Bowtie2

### 3.2.1 Instalação

Vamos mapear as reads contra o genoma humano e excluir essas reads usando Bowtie2.

In [ ]:
# Instalar Bowtie2
!conda install -c bioconda bowtie2 

  utils.DeprecatedIn35,
| / - \ | / - \ | / done
Solving environment: \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - bowtie2


The following packages will be SUPERSEDED by a higher-priority channel:

  openssl            conda-forge::openssl-1.1.1o-h166bdaf_0 --> pkgs/main::openssl-1.1.1o-h7f8727e_0


Proceed ([y]/n)? y

Preparing transaction: | done
Verifying transaction: - done
Executing transaction: | done


### 3.2.2 Contaminantes humanos

Não é possível mapear o arquivo .FASTQ de *Phrynomedusa vanzolini* diretamente contra o arquivo .FASTA humano. Primeiro, é necessário indexar o arquivo .FASTA. 

Obs. Os arquivos já indexados podem ser alternativamente baixados em http://ftp1.babraham.ac.uk/ftpusr46/FastQ_Screen_Genomes/Human/ (por algum motivo, o download anda lento nos últimos dias)

In [ ]:
# Cria um index para o .FASTA humano (não será necessário rodar esse código novamente uma vez que eu já tiver os arquivos .bt2)
!bowtie2-build /content/drive/MyDrive/Doutorado/Project/Tutoriais/fastqscreen_reference_genomes/Human/human.fas /content/drive/MyDrive/Doutorado/Project/Tutoriais/fastqscreen_reference_genomes/Human/human

Settings:
  Output files: "/content/drive/MyDrive/Doutorado/Project/Tutoriais/fastqscreen_reference_genomes/Human/human.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /content/drive/MyDrive/Doutorado/Project/Tutoriais/fastqscreen_reference_genomes/Human/human.fas
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:48
Calculating joined length
Writing header
Reserving space for joined string
tcmalloc: large alloc 3137306624 bytes == 0x564ed90d8000 @  0x7fe5b5f69887 0x564ed2b88949 0x564ed2b8a082 0x564ed2b5a2e7 0x564ed2b5c3c2 

Agora que temos a referência indexada, podemos rodar o Bowtie2. Os seguintes parâmetros serão usados:

- --very-fast-local: algoritmo pouco sensível mas rápido
- -x: diretório onde a referência humana indexada se encontra
- -U: diretório onde o .FASTQ de *Phrynomedusa* se encontra
- -S: diretório onde o output .SAM será colocado

In [ ]:
# Mapear reads de Phrynomedusa contra o genoma humano
!bowtie2 --very-fast-local -x /content/drive/MyDrive/Doutorado/Project/Tutoriais/fastqscreen_reference_genomes/Human/human -U /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing3_tally/output3_pvanzo_tally.fastq -S /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing4b_bowtie2/preprocessing4b

57576851 reads; of these:
  57576851 (100.00%) were unpaired; of these:
    56733038 (98.53%) aligned 0 times
    437059 (0.76%) aligned exactly 1 time
    406754 (0.71%) aligned >1 times
1.47% overall alignment rate


O output de Bowtie2 para o genoma humano foi:

```
57576851 reads; of these:
  57576851 (100.00%) were unpaired; of these:
    56733038 (98.53%) aligned 0 times
    437059 (0.76%) aligned exactly 1 time
    406754 (0.71%) aligned >1 times
1.47% overall alignment rate
```

Agora vamos extrair somente as reads não contaminadas, ou seja, aquelas que não foram mapeadas contra o genoma humano. Para isso, utilizaremos o SAMTools, com os seguintes argumentos:

- -b: output em formato BAM
- -f 4: mantém as reads não mapeadas
- -F 4: mantém as reads mapeadas

In [ ]:
# Instalar SAMTools
!conda install -c bioconda samtools 

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [ ]:
# Extrair as reads não mapeadas contra o genoma humano (não-contaminadas)
!samtools view -b -f 4 /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing4b_bowtie2/preprocessing4b.sam -@ 20 > /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing4b_bowtie2/bowtie2_nao_contaminados_pvanzo_human.bam

In [ ]:
# Extrair as reads mapeadas contra o genoma humano (contaminadas)
!samtools view -b -F 4 /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing4b_bowtie2/preprocessing4b.sam -@ 20 > /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing4b_bowtie2/bowtie2_contaminados_pvanzo_human.bam

In [ ]:
# Visualizar as primeiras linhas do arquivo de não contaminados .BAM 
!samtools view /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing4b_bowtie2/bowtie2_nao_contaminados_pvanzo_human.bam | head

trn_79	4	*	0	0	*	*	0	0	TCATGGCCAAAAGTTTTGAGAAT	AAAAAEEEEEEEEEEEEEEEEEE	YT:Z:UU
trn_80	4	*	0	0	*	*	0	0	TCATTCTCAAAACTTTTGGCCAT	AAAAAEEEEEEEEEEEEEEEEEE	YT:Z:UU
trn_91	4	*	0	0	*	*	0	0	CGTAAGAACTCAAAAGAAAGC	AAAAAEEEEEEEEEEEEEEEE	YT:Z:UU
trn_126	4	*	0	0	*	*	0	0	CATTCTCAAAACTTTTGGCCAT	AAAAAEEEEEEEEEEEEEEEEE	YT:Z:UU
trn_130	4	*	0	0	*	*	0	0	CGTAACAACTCAAAAGAAAGC	AAAAAEEEEEEEEEEEEEEEE	YT:Z:UU
trn_132	4	*	0	0	*	*	0	0	AACGCGTAAGAACTCAAAAGAAA	AAAAAEEEEEEEEEEEEEEEEEE	YT:Z:UU
trn_134	4	*	0	0	*	*	0	0	CATGGCCAAAAGTTTTGAGAAT	AAAAAEEEEEEEEEEEEEEEEE	YT:Z:UU
trn_148	4	*	0	0	*	*	0	0	TATCATTCTCAAAACTTTTGGCCAT	AAAAAEEEEEEEEEEEEEEEEEEEE	YT:Z:UU
trn_151	4	*	0	0	*	*	0	0	AACGCGTAAGAACTCAAAAGAA	AAAAAEEEEEEEEEEEEEEEEE	YT:Z:UU
trn_163	4	*	0	0	*	*	0	0	AACGCGTAACAACTCAAAAGAAA	AAAAAEEEEEEEEEEEEEEEEEE	YT:Z:UU


Agora, vamos converter o .BAM de reads não-contaminadas (i.e., não mapeadas contra o genoma humano) em .FASTQ.gz usando o programa **bedtools**. Isso é necessário, pois a próxima etapa removendo contaminantes de E. coli utiliza como input um arquivo em formato .FASTQ.gz

In [8]:
# Instalar bedtools
!conda install -c bioconda bedtools 

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - bedtools


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bedtools-2.30.0            |       h7d7f7ad_2        17.9 MB  bioconda
    bzip2-1.0.8                |       h7b6447c_0          78 KB
    openssl-1.1.1o             |       h7f8727e_0         2.5 MB
    ------------------------------------------------------------
                                           Total:        20.5 MB

The following NEW packages will be INSTALLED:

  bedtools           bioconda/linux-64::bedtools-2.30.0-h7d7f7ad_2
  bzip2              pkgs/main/linux-64::bzip2-1.0.8-h7b6447c_0

The following packages will be SUPERSEDED by a higher-priority channel:

  openssl            conda-forge::o

In [ ]:
# Converter .BAM em .FASTQ
!bedtools bamtofastq -i /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing4b_bowtie2/bowtie2_nao_contaminados_pvanzo_human.bam -fq /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/preprocessing4b_bowtie2/bowtie2_nao_contaminados_pvanzo_human.fastq

### 3.2.3 Contaminantes de E. coli

# 4 Genome Assembly 

Realizado no MITObim do Docker.



---



# 2 FASTQ SCREEN

## 2.1 Installation

In [ ]:
!conda install -c bioconda fastq-screen


Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.4
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /us

## 2.2 Downloading contaminant genomes

In [ ]:
# Não vale a pena baixar toda vez devido à demora de baixar os genomas... melhor fazer upload dos contaminantes em uma pasta do Drive
# !fastq_screen --get_genomes

Note que o caminho da pasta baixada no passo anterior é: /content/FastQ_Screen_Genomes

In [ ]:
!ls -l

total 57116
drwx------ 6 root root     4096 Jan 21 12:09 drive
drwxr-xr-x 3 root root     4096 Jan 21 12:11 ftp1.babraham.ac.uk
-rwxr-xr-x 1 root root 58468498 Jun  7  2018 Miniconda3-4.5.4-Linux-x86_64.sh
drwxr-xr-x 1 root root     4096 Jan  7 14:33 sample_data


In [ ]:
# Copiar arquivo de configuração para o diretório atual 'content'
!cp FastQ_Screen_Genomes/fastq_screen.conf .

cp: cannot stat 'FastQ_Screen_Genomes/fastq_screen.conf': No such file or directory


## 2.3 Testing dataset

In [ ]:
# Cheque o diretório atual
!pwd

/content


### 2.3.1 Downloading

In [ ]:
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastq_screen/fastq_screen_test_dataset.tar.gz

--2022-01-21 12:12:01--  https://www.bioinformatics.babraham.ac.uk/projects/fastq_screen/fastq_screen_test_dataset.tar.gz
Resolving www.bioinformatics.babraham.ac.uk (www.bioinformatics.babraham.ac.uk)... 149.155.133.4
Connecting to www.bioinformatics.babraham.ac.uk (www.bioinformatics.babraham.ac.uk)|149.155.133.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147199 (144K) [application/x-gzip]
Saving to: ‘fastq_screen_test_dataset.tar.gz’

fastq_screen_test_d 100%[===================>] 143.75K   510KB/s    in 0.3s    

2022-01-21 12:12:01 (510 KB/s) - ‘fastq_screen_test_dataset.tar.gz’ saved [147199/147199]



In [ ]:
!tar xvzf fastq_screen_test_dataset.tar.gz

fastq_screen_test_dataset/
fastq_screen_test_dataset/fqs_test_dataset_screen.txt
fastq_screen_test_dataset/README.txt
fastq_screen_test_dataset/fqs_test_dataset_screen.png
fastq_screen_test_dataset/fqs_test_dataset_screen.html
fastq_screen_test_dataset/fqs_test_dataset.fastq.gz


In [ ]:
# Copiar arquivo fastq de teste para o diretório atual
!cp fastq_screen_test_dataset/fqs_test_dataset.fastq.gz .

### 2.3.2 FASTQ SCREEN configuration file

Existem 3 informações que você precisa descobrir para editar o arquivo fastq_screen.conf:



1.   O caminho do bowtie2
2.   O número de núcleos do processador do Google Colab
3.   O caminho dos genomas contaminantes



In [ ]:
# Verify the path of bowtie2
!which bowtie2

/usr/local/bin/bowtie2


In [ ]:
# Verify the number of threads in Google Colab's CPU
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

2

Para verificar o caminho do diretório dos genomas de contaminantes, basta entrar na pasta 'FastQ_Screen_Genomes', clicar nos três pontinhos à direita do nome de algum contaminante aqui no Colab, e selecionar "Copy path". Em seguida, deve-se adicionar ao final do path o index do bowtie2. Por exemplo:

- Se a pasta Human está em /content/FastQ_Screen_Genomes/Human 

- E se dentro da pasta há os arquivos Homo_sapiens.GRCh38.1.bt2 e Homo_sapiens.GRCh38.2.bt2

- Logo, deve-se digitar 'DATASET Human /content/FastQ_Screen_Genomes/Human/Homo_sapiens.GRCh38' dentro do arquivo fastq_screen.conf

Os próximos passos devem ser feitos fora do Colab, porque não dá para usar o Nano para editar texto aqui:

- Clique no símbolo dos 3 riscos à esquerda do Colab. 
- Depois, clique em Files
- Baixe o arquivo /content/fastq_screen.conf
- Edite no seu computador local:


```
# Edições necessárias no fastq_screen.conf

...

BOWTIE2 /usr/local/bin/bowtie2

...

THREADS		2

...

DATABASE	Human	/content/FastQ_Screen_Genomes/Human/Homo_sapiens.GRCh38

```

### 2.3.3 Mapping against contaminants

In [ ]:
!fastq_screen --conf /content/drive/MyDrive/ColabNotebooks/fastqscreen/test_dataset/fastq_screen.conf --aligner bowtie2 fastq_screen_test_dataset.tar.gz

Using fastq_screen v0.14.0
Reading configuration from '/content/drive/MyDrive/ColabNotebooks/fastqscreen/test_dataset/fastq_screen.conf'
Using '/usr/local/bin/bowtie2' as Bowtie 2 path
Skipping DATABASE 'Human' since no Bowtie 2 index was found at '/content/FastQ_Screen_Genomes/Human/Homo_sapiens.GRCh38'
Skipping DATABASE 'Mouse' since no Bowtie 2 index was found at '/content/FastQ_Screen_Genomes/Mouse/Mus_musculus.GRCm38'
Skipping DATABASE 'Rat' since no Bowtie 2 index was found at '/content/FastQ_Screen_Genomes/Rat/Rnor_6.0'
Skipping DATABASE 'Ecoli' since no Bowtie 2 index was found at '/content/FastQ_Screen_Genomes/E_coli/Ecoli'
Skipping DATABASE 'Vectors' since no Bowtie 2 index was found at '/content/FastQ_Screen_Genomes/Vectors/Vectors'
Skipping DATABASE 'Adapters' since no Bowtie 2 index was found at '/content/FastQ_Screen_Genomes/Adapters/Contaminants'
Using 2 threads for searches
No reference genomes were configured, please adjust configuration.


In [ ]:
import IPython
IPython.display.HTML(filename='/content/fastq_screen_test_dataset.tar_screen.html')

FileNotFoundError: ignored

Note que durante o 'testing dataset', estávamos no diretório '/content'. Antes de irmos brincar com archival DNA de verdade, vamos criar uma nova pasta chamada 'teste_output' para jogarmos o output dos testes em 2.3. Isso é necessário, pois o FASTQ SCREEN não roda direito se arquivos de output dele já estiverem naquele diretório.


In [ ]:
!mkdir teste_output

In [ ]:
!mv /content/fastq_screen_test_dataset.tar_screen.html /content/teste_output/fastq_screen_test_dataset.tar_screen.html

In [ ]:
!mv /content/fastq_screen_test_dataset.tar_screen.png /content/teste_output/fastq_screen_test_dataset.tar_screen.png

In [ ]:
!mv /content/fastq_screen_test_dataset.tar_screen.txt /content/teste_output/fastq_screen_test_dataset.tar_screen.txt

## 2.4 Testing with archival DNA from *Boana claresignata*

Os seguintes arquivos são necessários:


1.   Arquivos FASTQ de músculo e fígado de *Boana claresignata*
2.   Arquivo FASTA + indexados de bowtie2 de genoma de referência (*Aplastodiscus*)
3.   Arquivos de genoma contaminante (dentro da pasta FastQ_Screen_Genomes)
4.   Arquivo de configuração do FASTQ SCREEN

Os arquivos 1, 2 e 4 estão no diretório /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020 do Drive de dani_ymn@usp.br. Se usar esse colab em outro e-mail, deve-se adaptar os caminhos de todos os arquivos.


In [ ]:
# Check current folder
!pwd

### 2.4.1 Arquivo 1: FASTQ de *B. claresignata*

Os arquivos FASTQ de músculo *B. claresignata* devem ser colocados no diretório /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_muscle, ao passo que os arquivos de fígado em /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_liver.

Veja que há pelo menos 4 arquivos de cada tecido:


```
# Arquivos de fígado
liver1: /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_liver/CH361L_cap_S1_L001_R1_001.fastq 
liver2: /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_liver/CH361L_cap_S1_L002_R1_001.fastq
liver3: /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_liver/CH361L_cap_S1_L003_R1_001.fastq
liver4: /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_liver/CH361L_cap_S1_L004_R1_001.fastq

# Arquivos de músculos
muscle1: /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_muscle/CH361M_cap_S36_L001_R1_001.fastq
muscle2: /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_muscle/CH361M_cap_S36_L002_R1_001.fastq
muscle3: /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_muscle/CH361M_cap_S36_L003_R1_001.fastq
muscle4: /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_muscle/CH361M_cap_S36_L004_R1_001.fastq
```

#### 3.4.1.1 Juntando (merge) arquivos FASTQ

Vamos juntar os .FASTQ de fígado usando cat do Linux.

In [ ]:
# Merge FASTQ files of liver1 and liver2
!cat /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_liver/CH361L_cap_S1_L001_R1_001.fastq /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_liver/CH361L_cap_S1_L002_R1_001.fastq > combined_CH361L_12.fastq

In [ ]:
# Merge FASTQ files of liver1+2 and liver3
!cat /content/combined_CH361L_12.fastq /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_liver/CH361L_cap_S1_L003_R1_001.fastq > combined_CH361L_123.fastq

In [ ]:
# Merge FASTQ files of liver1+2+3 and liver4
!cat /content/combined_CH361L_123.fastq /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_liver/CH361L_cap_S1_L004_R1_001.fastq > combined_CH361L_1234.fastq

In [ ]:
!head /content/combined_CH361L_1234.fastq

Agora vamos juntar os .FASTQ de músculo.

In [ ]:
# Merge FASTQ files of muscle1 and muscle2
!cat /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_muscle/CH361M_cap_S36_L001_R1_001.fastq /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_muscle/CH361M_cap_S36_L002_R1_001.fastq > combined_CH361M_12.fastq

In [ ]:
# Merge FASTQ files of muscle1+2 and muscle3
!cat /content/combined_CH361M_12.fastq /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_muscle/CH361M_cap_S36_L003_R1_001.fastq > combined_CH361M_123.fastq

In [ ]:
# Merge FASTQ files of muscle1+2+3 and muscle4
!cat /content/combined_CH361M_123.fastq /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/claresignata_muscle/CH361M_cap_S36_L004_R1_001.fastq > combined_CH361M_1234.fastq

In [ ]:
!head /content/combined_CH361M_1234.fastq

Agora vamos juntar músculo e fígado em um único arquivo FASTQ.

In [ ]:
!cat /content/combined_CH361L_1234.fastq /content/combined_CH361M_1234.fastq > combined_CH361ML_1234.fastq

#### 3.4.1.2 Trimar reads menores de 30pb

Realizar uma rodada de trimagem usando Cutadapt, com adaptador 1 (AGATCGGAAGAGCACACGTCTGAACTCCAGTCA) de Illumina HiSeq, depois com adaptador 2 (AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT).

Mais informações de cutadapt em: https://cutadapt.readthedocs.io/en/stable/recipes.html

In [ ]:
!conda install -c bioconda cutadapt

In [ ]:
!cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -m 30 /content/combined_CH361ML_1234.fastq | cutadapt -a AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -> output_cutadapt30bp.fastq

#### 2.4.1.3 Trimar duplicatas de PCR


Vamos remover duplicatas de PCR com Tally do Kraken.

In [ ]:
!conda install -c bioconda reaper

In [ ]:
!tally -i /content/output_cutadapt30bp.fastq -o output_tally_CH361ML.fastq --nozip --with-quality

### 2.4.2 Arquivo 2: Genoma de referência 

Para gerar os indexados de bowtie2 do arquivo 2, o arquivo FASTA do mitogenoma de Aplastodiscus deve estar em /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/fasta/pithecopus_megacephalus_whole_mitogenome_MG772558.fasta. Depois, basta digitar:

In [ ]:
# Criar indexamento de bowtie2 a partir do arquivo FASTA
!bowtie2-build /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/fasta/pithecopus_megacephalus_whole_mitogenome_MG772558.fasta /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/pithecopus_index
!bowtie2-inspect pithecopus_index


Settings:
  Output files: "/content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/pithecopus_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /content/drive/MyDrive/Doutorado/Project/C6_phrynomedusa/phrynomedusa/reference_genome/fasta/pithecopus_megacephalus_whole_mitogenome_MG772558.fasta
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00

Note que os seguintes arquivos de Pithecopus foram criados no diretório atual /content

1.   /content/pithecopus_index.1.bt2
2.   /content/pithecopus_index.2.bt2
3.   /content/pithecopus_index.3.bt2
4.   /content/pithecopus_index.4.bt2
5.   /content/pithecopus_index.rev.1.bt2
6.   /content/pithecopus_index.rev.2.bt2

### 2.4.3 Arquivo de configuração

O novo arquivo de configuração ficará salvo em /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020. Além de mapear contra o genoma de Aplastodiscus, mapeei também contra o genoma humano, de E. coli, de vetor e adaptadores. Deve-se adaptar o arquivo de configuração da seguinte maneira:



```
# fastq_screen.conf para Lyra et al. (2020)
...
BOWTIE2 /usr/local/bin/bowtie2
...
THREADS		2
...
DATABASE	Human	/content/FastQ_Screen_Genomes/Human/Homo_sapiens.GRCh38
...
DATABASE	Ecoli	/content/FastQ_Screen_Genomes/E_coli/Ecoli
...
DATABASE	Vectors	/content/FastQ_Screen_Genomes/Vectors/Vectors
...
DATABASE	Adapters	/content/FastQ_Screen_Genomes/Adapters/Contaminants
...
DATABASE	Frog	/content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/aplastodiscus/sequence
```

PS. 'Frog' se refere ao genoma de Aplastodiscus, a partir do qual esperamos que muitas reads de *B. claresignata* mapeiem contra.


### 2.4.4 FASTQ SCREEN: Análise Exploratória

#### 2.4.4.1 FASTQ sem trimar

In [ ]:
# Run FASTQ SCREEN for the combined Muscle fastq
!fastq_screen --conf /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/fastq_screen.conf --aligner bowtie2 /content/combined_CH361M_1234.fastq

In [ ]:
# Run FASTQ SCREEN for the combined liver fastq
!fastq_screen --conf /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/fastq_screen.conf --aligner bowtie2 /content/combined_CH361L_1234.fastq

In [ ]:
# Run FASTQ SCREEN for the combined liver fastq
!fastq_screen --conf /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/fastq_screen.conf --aligner bowtie2 /content/combined_CH361ML_1234.fastq

In [ ]:
import IPython
IPython.display.HTML(filename='/content/combined_CH361ML_1234_screen.html')

In [ ]:
# Check average read length and number of reads WITHOUT TRIMMING
!awk 'NR%4 == 2 {lengths[length($0)]++} END {for (l in lengths) {print l, lengths[l]}}' /content/combined_CH361ML_1234.fastq 


#### 2.4.4.2 FASTQ sem adaptadores e >30bp (CUTADAPT)

Depois de remover as reads com menos de 30 pb em Trimmomatic:

In [ ]:
# Check average read length and number of reads WITH TRIMMING (>30bp)
!fastq_screen --conf /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/fastq_screen.conf --aligner bowtie2 /content/output_cutadapt30bp.fastq

In [ ]:
import IPython
IPython.display.HTML(filename='/content/output_cutadapt30bp_screen.html')

#### 2.4.4.3 FASTQ sem duplicatas de PCR (TALLY)

Depois de remover as reads de duplicatas de PCR usando TALLY do Kraken Tools:

In [ ]:
!!fastq_screen --conf /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/fastq_screen.conf --aligner bowtie2 /content/output_tally_CH361ML.fastq

In [ ]:
import IPython
IPython.display.HTML(filename='/content/output_tally_CH361ML_screen.html')

#### 2.4.4.4 FASTQ sem contaminantes (FASTQ SCREEN)

In [ ]:
!cp /content/output_tally_CH361ML.fastq .

In [ ]:
!fastq_screen --tag --filter 00003 --aligner bowtie2 --conf /content/drive/MyDrive/ColabNotebooks/fastqscreen/lyra2020/fastq_screen.conf /content/output_tally_CH361ML.fastq